In [11]:
import torch
import torch.nn as nn
import math
import torch.utils.model_zoo as model_zoo

In [20]:
def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


![image](https://user-images.githubusercontent.com/44194558/153380033-a346c15a-a1ae-4701-9205-af72bfe77062.png)

<br/>

각 layer는 이전 layer의 출력을 입력을 받음 
 - 이전 layer의 출력은 현재 layer의 첫 번째 block의 첫 번째 conv 연산을 통해 downsampling됨 (채널 축소)

각 layer를 구성하는 block에서 첫 번째 conv 연산은 이전 block의 출력을 입력으로 받음

 - 이전 block의 출력은 현재 block의 첫 번째 conv 연산을 통해 downsampling됨 (채널 축소)

In [21]:
# layer를 구성하는 block (각 layer에서 지정된 수 만큼 block을 쌓음)
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.sigmoid = nn.Sigmoid()
        self.downsample = downsample
        self.stride = stride

        ## 1x1 -> 3x3 -> 1x1 conv 연산이 하나의 ResNet block을 구성함
        # 1x1 conv
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)  
        self.bn1 = nn.BatchNorm2d(planes)

        # 3x3 conv
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        # 1x1 conv (expansion)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        
        # output size에 적합한 pooling
        if planes == 64:  # layer1 (56x56)
            self.globalAvgPool = nn.AvgPool2d(56, stride=1)
        elif planes == 128:  # layer2 (128x128)
            self.globalAvgPool = nn.AvgPool2d(28, stride=1)
        elif planes == 256:  # layer3 (256x256)
            self.globalAvgPool = nn.AvgPool2d(14, stride=1)
        elif planes == 512:  # layer4 (512x512)
            self.globalAvgPool = nn.AvgPool2d(7, stride=1)
        
        # SE block의 Excitation 연산용 F.C layer
        self.fc1 = nn.Linear(in_features=planes * 4, out_features=round(planes / 4))  # C -> C/r
        self.fc2 = nn.Linear(in_features=round(planes / 4), out_features=planes * 4)  # C/r -> C
        
    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        original_out = out  # feature
      
        # Squeeze
        out = self.globalAvgPool(out)
        out = out.view(out.size(0), -1)

        # Excitation
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        out = out.view(out.size(0), out.size(1), 1, 1)
        
        # Feature recallibration
        out = out * original_out
        
        # Skip-conn
        out += residual
        out = self.relu(out)

        return out

![image](https://user-images.githubusercontent.com/44194558/153379557-2ea2c618-2945-4f6e-8d8b-61236ec962e7.png)

<br/>

Residual block은(빨간색) 기존의 ResNet과 동일

Skip-conn전에 SE block이 (파란색) 추가됨


In [22]:
class SENet(nn.Module):

    def __init__(self, block, layers, num_classes=1000):
        self.inplanes = 64
        super(SENet, self).__init__()

        # Conv1
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Conv2_x ~ Conv5_x (layers=[3, 4, 6, 3])
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)

        # Classifier
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))

            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
    
    # Residual+SE block 생성 
    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            # 3개의 conv 연산으로 구성된 모든 block에서 첫 번째 연산은 증가된 차원(채널)을 입력으로 받음
            # skip-conn을 위해 차원을 맞춰줄 수 있도록 expansion
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
                )
            
            layers = []
            layers.append(block(self.inplanes, planes, stride, downsample))  # layer의 첫 번째 block
            self.inplanes = planes * block.expansion  # 증가된 차원

            # 각 layer의 두 번째 block은 증가된 채널을 입력으로 받음
            for i in range(1, blocks):
                layers.append(block(self.inplanes, planes))

            return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        # classification
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [23]:
def se_resnet_50(pretrained=False, **kwargs):
    """Constructs a ResNet-50 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = SENet(Bottleneck, [3, 4, 6, 3], **kwargs)
    
    return model

In [24]:
 se_resnet_50(pretrained=False)

SENet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (sigmoid): Sigmoid()
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1,